In [7]:
import requests                                             #python3 -m pip install requests
from urllib.request import Request, urlopen 
import urllib.request
import ssl
import pickle
import shelve
import ctypes
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from enum import Enum
import time
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, timedelta

In [8]:
def requestToken():
    url_base='https://'+ '157.55.183.132' +'/sba_hub/API/public/index.php/api/v1/hubapi'
    r = requests.Response
    try:
        HUB_Service_Token = '/auth/device'
        REQUEST_TIMEOUT = (3, 3)  # 3 second to connect, 3 second to receive response
        tokenURL = url_base + HUB_Service_Token
        tokenInfo = {'Administrador ': '2161'}
        r = requests.post(url=tokenURL, json=tokenInfo, timeout=REQUEST_TIMEOUT,verify=False)  # add verify False SSL
    except Exception as e:
        print('Exception: ' + str(e))
    return r.status_code

In [11]:
class BearerAuth(requests.auth.AuthBase):
    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        r.headers["authorization"] = "Bearer " + self.token
        return r

def GetRecords(user,password,hub,from_date,to_date,sap="",device="",site=""):
    url = 'https://'+hub+'/sba_hub/API/public/index.php/api/v1/hubapi/auth/login'
    payload = {"userid": user, "passwd": password}
    login_response = requests.post(url, data=payload,verify=False)
    token = login_response.json()['token']
    POST_URL = 'https://'+ hub+ '/sba_hub/API/public/index.php/api/v1/hubapi/records/all'
    payload = {"from_date": from_date,"to_date": to_date,"sap": sap,"device": device,"site": site}
    REQUEST_TIMEOUT_POST_PENDING = (3, 6)
    users_records = requests.post(url=POST_URL, json=payload, timeout=REQUEST_TIMEOUT_POST_PENDING,auth=BearerAuth(token), verify=False)  # add verify False SSL
    USER_TYPE = ['Empleado','Visita','NOT_DEFINED','NOT_DEFINED']
    AUTORIZED_ACCESS = ['Si','No','No']
    CUBREBOCAS = ['Incorrecto','Correcto']
    ESTATUS_MONITOR = ['Enviado','No Enviado','NOT_DEFINEED_3','ID Seguimiento inválido','NOT_DEFINED_5']
    user_record_list = []
    for user_record in users_records.json():
        user_record_json ={}
        user_record_json['#Empleado_ID'] = user_record['sap_number']
        user_record_json['Nombre'] = user_record['users']['lastname'] + ' '+ user_record['users']['surname'] + ' '+ user_record['users']['name']
        user_record_json['Dispositivo'] = user_record['devices']['mac_address']
        user_record_json['Alias'] = user_record['devices']['device_type']
        user_record_json['ID_Seguimiento'] = user_record['monitor_id']
        user_record_json['Tipo_Usuario'] = USER_TYPE[user_record['user_type']-1]
        user_record_json['Autorizado?'] = AUTORIZED_ACCESS[user_record['authorized_access']-1]
        user_record_json['Temperatura'] = user_record['temperature']
        user_record_json['Fecha_Registro'] = user_record['date_time']
        user_record_json['Cubrebocas?'] = CUBREBOCAS[user_record['mask']]
        user_record_json['Estatus_Monitor_FCS'] = ESTATUS_MONITOR[user_record['status']-1]
        user_record_json['Fecha_Envio_Monitor_FCS'] = user_record['to_fcs']
        user_record_list.append(user_record_json)        
    data_set = pd.DataFrame.from_records(user_record_list)
    return data_set

In [12]:
user = 'admin-SBA'                  #El user del HUB
password = 'SB4p4ssw0rd*21'         #El password del HUB
hub = '10.200.19.17'                #Solo la IP del HUB o el Alias del HUB arnelec o como le llamen ejemplo  https://IP_DEL_HUB/sba_hub
day_to_report = str(date.today() - timedelta(days=1))
from_date = day_to_report + ' 00:00:00'
to_date = day_to_report + ' 23:59:59'
data_set = GetRecords(user=user,password=password,hub=hub,from_date=from_date,to_date=to_date)
data_set.to_excel(day_to_report +'_'+hub+ '_Registros.xlsx',sheet_name='registros', index=False)
data_set.head(15)

,#Empleado_ID,Nombre,Dispositivo,Alias,ID_Seguimiento,Tipo_Usuario,Autorizado?,Temperatura,Fecha_Registro,Cubrebocas?,Estatus_Monitor_FCS,Fecha_Envio_Monitor_FCS
0,970292,FLORES FLORES MARTHA,48:B0:2D:3D:9B:CF,CUPRO35,279652,Empleado,Si,35.3,2022-10-08 23:23:32,Incorrecto,Enviado,2022-10-08 23:24:01
1,918561,GARCIA GOMEZ HUGO,48:B0:2D:3D:69:FC,CUPRO33,296602,Empleado,Si,36.3,2022-10-08 23:22:53,Incorrecto,Enviado,2022-10-08 23:24:01
2,991709,LOZANO MARTINEZ RAUL,48:B0:2D:3D:7B:D1,CUPRO31,490376,Empleado,Si,36.5,2022-10-08 23:16:28,Incorrecto,Enviado,2022-10-08 23:18:01
3,918258,ORTIZ VAZQUEZ FRANCISCO,48:B0:2D:3D:99:9A,CUPRO34,268857,Empleado,Si,35.3,2022-10-08 23:16:14,Incorrecto,Enviado,2022-10-08 23:18:01
4,918463,GARCIA GUERRERO GERARDO CARLOS ALBERTO,48:B0:2D:3D:9B:CF,CUPRO35,364659,Empleado,Si,35.3,2022-10-08 23:15:44,Incorrecto,Enviado,2022-10-08 23:16:01
5,977021,REYNA REYNA NATALIA,48:B0:2D:3D:69:FC,CUPRO33,352336,Empleado,Si,35.3,2022-10-08 23:13:59,Incorrecto,Enviado,2022-10-08 23:14:01
6,918375,DE LIRA DE LIRA IGNACIO,48:B0:2D:3D:9B:CF,CUPRO35,352385,Empleado,Si,35.3,2022-10-08 23:13:56,Incorrecto,Enviado,2022-10-08 23:14:01
7,984515,PIÑA BUENDIA MONICA FLOR,48:B0:2D:3D:99:8A,CUPRO25,336486,Empleado,Si,36.5,2022-10-08 23:13:45,Incorrecto,Enviado,2022-10-08 23:14:01
8,991763,LOPEZ LOPEZ ISAURA NOEMY,48:B0:2D:3D:69:FC,CUPRO33,490363,Empleado,Si,36.0,2022-10-08 23:13:36,Incorrecto,Enviado,2022-10-08 23:14:01
9,992433,BRIONES ALVARADO ERIKA LILIANA,48:B0:2D:3D:99:8A,CUPRO25,505883,Empleado,Si,35.7,2022-10-08 23:13:34,Incorrecto,Enviado,2022-10-08 23:14:01
